# ML классификатор 

In [348]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.models import load_model
import numpy as np
import pandas as pd

In [349]:
# Загружаем файл с разделителем ';' в DF
df = pd.read_csv(fr'nn_features_and_target.csv', delimiter=';')
# df.columns = df.columns.astype(str)
df['22'] = df.apply(lambda x: 1 if x['21'] == 0 else 0, axis=1)
print(df.to_string(max_rows=4, max_cols=24))

           0       1       2       3       4       5       6       7      8      9     10     11     12     13     14     15     16     17     18     19      20  21  22
0     142848  134054  133546  123646  123022  109730  102368   90738  83022  58726 -32210 -19654  -4732   9432  27792  35356  66110  76092  92154  94122  121158   0   1
1     140820  140754  132154  131646  121752  121132  108072  100666  89428  81100  57282 -29934 -17318  -2042  12132  31476  38872  70066  80016  96062   98026   1   0
...      ...     ...     ...     ...     ...     ...     ...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...    ...    ...    ...     ...  ..  ..
2328   52374   52138   49356   49164   45542   44934   39138   36062  30764  22292  -3932  15276  33322  44912  54068  57828  58646  61560  61844  61878   61916   1   0
2329   35044   34808   32070   31918   28884   28572   23520   21854  18306  13036  -4612   3870  18706  27552  35436  38446  39230  42124  42408  42442   

In [350]:
# Преобразование в DataSet
dataset = df.values  # Dataframe преобразуем в Dataset для Keras
dataset = dataset.astype(np.float32)  # Смена типа для корректной работы Keras
# Все, что стоит перед запятой, относится к строкам массива, а все, что стоит после запятой,
# относится к столбцам массивов.
X = dataset[:, 0:21]  # Срез массива по фичам
y = dataset[:, 21:23]  # Срез массива по labels

# Разделение данных на обучающую, тестовую и валидационную выборки
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.15,
    random_state=3,
    shuffle=True
)

# scaler = StandardScaler()
# scaler = MinMaxScaler() 
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

print(len(X_train[0]))
# print(X_train)
# print('\n', X_test)

21


In [351]:
# Архитектура сети
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# for _ in range(3):
#     model.add(Dense(512, activation='relu', ))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.1))
# model.add(Dense(y_train.shape[1], activation='softmax'))

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))  # , kernel_regularizer=l2(0.001)
model.add(BatchNormalization())
model.add(Dropout(0.3))
for _ in range(5):
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))  # , kernel_regularizer=l2(0.001)
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Параметры модели
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

c:\Users\Alkor\PycharmProjects\pj1_ML_classifier_RTS_SPAN_CC\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [352]:
# Определение функции обратного вызова EarlyStopping
early_stopping = EarlyStopping(
    # monitor='val_loss',      # Мониторим валидационную потерю
    monitor='val_accuracy',  # Мониторим валидационную точность
    patience=2000,           # Количество эпох без улучшения до остановки
    verbose=0,               # Вывод информации в процессе обучения
    restore_best_weights=True  # Восстанавливаем веса модели с наилучшей валидационной потерей
)

# Определение функции обратного вызова ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    filepath='best_model.keras',   # Путь для сохранения модели
    # monitor='val_loss',            # Мониторим валидационную потерю
    monitor='val_accuracy',        # Мониторим валидационную точность
    save_best_only=True,           # Сохраняем только лучшую модель
    verbose=1                      # Вывод информации в процессе сохранения
)

# Обучение модели с использованием функций обратного вызова
history = model.fit(
    X_train, y_train,
    epochs=10000,               # Максимальное количество эпох
    batch_size=512,            # Размер мини-пакета
    validation_split=0.15,      # Доля данных для валидации
    callbacks=[early_stopping, model_checkpoint]  # Список функций обратного вызова
)


Epoch 1/10000
3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5080 - loss: 3.7540
Epoch 1: val_accuracy improved from -inf to 0.50168, saving model to best_model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.5042 - loss: 3.7511 - val_accuracy: 0.5017 - val_loss: 3.2927
Epoch 2/10000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4914 - loss: 3.7334
Epoch 2: val_accuracy did not improve from 0.50168
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4912 - loss: 3.7330 - val_accuracy: 0.5017 - val_loss: 3.2917
Epoch 3/10000
3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5117 - loss: 3.6357
Epoch 3: val_accuracy did not improve from 0.50168
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5136 - loss: 3.6249 - val_accuracy: 0.4983 - val_loss: 3.2903
Epoch 4/10000
3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5138 - loss: 3.6312
Epoch 4: val_accuracy did not improve from 0.50168
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5150 - loss: 3.6354 - 

In [353]:
del model  # deletes the existing model
model = load_model('best_model.keras')  # Загрузка сохраненной лучшей модели

In [354]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4831 - loss: 3.1780  
Test Accuracy: 0.5029


In [355]:
predictions = model.predict(X_test)

# Пример вывода вероятностей для примеров из тестовой выборки
print(predictions)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[0.60646564 0.39353433]
 [0.5752667  0.42473328]
 [0.53488606 0.46511394]
 [0.61214656 0.38785338]
 [0.60475546 0.39524457]
 [0.560736   0.439264  ]
 [0.5355686  0.46443135]
 [0.605218   0.39478207]
 [0.47019434 0.5298056 ]
 [0.5908728  0.40912718]
 [0.48072335 0.5192767 ]
 [0.48875216 0.5112478 ]
 [0.5723288  0.42767128]
 [0.47596595 0.5240341 ]
 [0.55496675 0.44503316]
 [0.4683984  0.5316016 ]
 [0.5810581  0.41894192]
 [0.56622005 0.43377995]
 [0.5876397  0.41236037]
 [0.49686414 0.5031358 ]
 [0.48986918 0.5101308 ]
 [0.51595217 0.4840479 ]
 [0.58554715 0.4144528 ]
 [0.58200824 0.4179917 ]
 [0.45382258 0.54617745]
 [0.5756402  0.42435974]
 [0.63328284 0.3667171 ]
 [0.5378294  0.4621706 ]
 [0.525586   0.47441402]
 [0.5897633  0.41023666]
 [0.46695983 0.53304017]
 [0.59385365 0.40614635]
 [0.48073083 0.5192692 ]
 [0.467077   0.5329229 ]
 [0.5093943  0.49060577]
 [0.49205709 0.507943  ]
 [0.5427483  0.45725176]
 [0.45548978 0.5445101 ]
 [0.572418

In [356]:
np.savetxt('predictions.txt', predictions, fmt='%.4f')
# print(predictions, fmt='%.4f')
# print(predictions('{:,.2f}'.format))

In [359]:
border = 0.6  # Пороговое значение вероятности по предсказанию

def profit_loss(u_p, d_p, u_r, d_r):
    if u_p > border and u_r == 1.0:
        return 1
    elif d_p > border and d_r == 1.0:
        return 1
    else:
        return 0

df = pd.DataFrame(predictions, columns=['up_predict', 'down_predict'])
df_y = pd.DataFrame(y_test, columns=['up_real', 'down_real'])
df = pd.merge(df, df_y, left_index=True, right_index=True)  # , suffixes=('_predict', '_real')
df = df[(df.up_predict > border) | (df.down_predict > border)]  # Отсекаем неудовлетворяющие строки
df['profit_loss'] = df.apply(lambda x: profit_loss(x.up_predict, x.down_predict, x.up_real, x.down_real), axis=1)
print(df.to_string(max_rows=20, max_cols=22))

     up_predict  down_predict  up_real  down_real  profit_loss
0      0.606466      0.393534      0.0        1.0            0
3      0.612147      0.387853      1.0        0.0            1
4      0.604755      0.395245      0.0        1.0            0
7      0.605218      0.394782      0.0        1.0            0
26     0.633283      0.366717      0.0        1.0            0
45     0.638561      0.361439      0.0        1.0            0
52     0.617110      0.382890      1.0        0.0            1
57     0.616288      0.383712      1.0        0.0            1
61     0.626763      0.373237      1.0        0.0            1
66     0.652628      0.347372      0.0        1.0            0
..          ...           ...      ...        ...          ...
259    0.619025      0.380975      0.0        1.0            0
262    0.644636      0.355364      1.0        0.0            1
266    0.612194      0.387806      0.0        1.0            0
273    0.610729      0.389271      0.0        1.0      

In [360]:
rez = df.profit_loss.sum() / len(df.profit_loss)
print(rez)

0.5510204081632653
